In [1]:
import sys
sys.path.append('/content/drive/MyDrive/SNLP/Extra')

In [2]:
import torch
from transformers import DistilBertTokenizerFast, DistilBertModel, DistilBertConfig, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
MAX_LENGTH = 512
BATCH_SIZE = 64

skip_training = False


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [3]:
def collate(batch):
    batch_sorted = sorted(batch, key=lambda x: x['input_ids'].size(0), reverse=True)

    input_ids = [item['input_ids'] for item in batch_sorted]
    attention_masks = [item['attention_mask'] for item in batch_sorted]
    labels = [item['label'] for item in batch_sorted]

    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask_padded = pad_sequence(attention_masks, batch_first=True, padding_value=0)

    labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)

    return input_ids_padded, attention_mask_padded, labels

In [4]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


In [5]:
import numpy as np

from sklearn.metrics import f1_score

def compute_f1(model, val_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for data in val_loader:
            inputs, mask, labels = data

            inputs = inputs.to(device)
            mask = mask.to(device)
            labels = labels.to(torch.float32).reshape(labels.size(0), 1).to(device)

            outputs = model(inputs, mask)
            predicted = torch.round(torch.sigmoid(outputs))

            all_preds.append(predicted.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)

    f1 = f1_score(all_labels, all_preds, average='binary')
    return f1



import numpy as np

def compute_accuracy(model, val_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for data in val_loader:
            inputs, mask, labels = data

            inputs = inputs.to(device)
            mask = mask.to(device)
            labels = labels.to(torch.float32).reshape(labels.size(0), 1).to(device)

            outputs = model(inputs, mask)
            predicted = torch.round(torch.sigmoid(outputs))

            all_preds.append(predicted.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)

    accuracy = np.mean(all_preds == all_labels)

    return accuracy


In [6]:
train_df = pd.read_csv('/content/drive/MyDrive/SNLP/Extra/data/train_2024.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/SNLP/Extra/data/dev_2024.csv')
test_df = pd.read_csv('/content/drive/MyDrive/SNLP/Extra/data/test_2024.csv')

train_df = train_df[train_df['label'].notna()]
dev_df = dev_df[dev_df['label'].notna()]

train_texts, train_labels = train_df['text'].tolist(), train_df['label'].tolist()
dev_texts, dev_labels = dev_df['text'].tolist(), dev_df['label'].tolist()
test_texts = test_df['text'].tolist()

trainset = TextDataset(train_texts, train_labels, tokenizer, MAX_LENGTH)
devset = TextDataset(dev_texts, dev_labels, tokenizer, MAX_LENGTH)

train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, collate_fn=collate)
dev_loader = DataLoader(devset, batch_size=BATCH_SIZE, collate_fn=collate)


In [7]:
class SentimentClassifier(nn.Module):
    def __init__(self):
        super(SentimentClassifier, self).__init__()
        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, 1)
    def forward(self, input_ids, attention_mask):
        pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        ).last_hidden_state[:, 0, :]
        output = self.drop(pooled_output)
        return self.out(output)

model = SentimentClassifier()
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-5, betas=(0.9, 0.98), eps=1e-9)
criterion = nn.BCEWithLogitsLoss()


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [9]:
epochs = 15
accumulation_steps = 2

from torch.cuda.amp import GradScaler, autocast

scaler = GradScaler()

print(f'Entering the training loop \n')

if not skip_training:
  for epoch in range(epochs):
    total_loss = 0.0
    correct = 0
    total = 0

    optimizer.zero_grad()
    model.train()

    for i, data in enumerate(train_loader):
        inputs, mask, labels = data

        inputs = inputs.to(device)
        mask = mask.to(device)
        labels = labels.to(device)

        with autocast():
            outputs = model(inputs, mask)
            loss = criterion(outputs, labels) / accumulation_steps

        scaler.scale(loss).backward()

        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        total_loss += loss.item() * accumulation_steps

        predicted = torch.round(torch.sigmoid(outputs))
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        if (i + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}, Batch {i + 1}, Loss: {loss.item():.4f}")

    epoch_loss = total_loss / len(train_loader)
    epoch_accuracy = correct / total * 100.0

    print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    val_accuracy = compute_accuracy(model, dev_loader, device)
    val_f1 = compute_f1(model, dev_loader, device)
    print(f"Validation accuracy: {val_accuracy:.2f}%")
    print(f"Validation F1: {val_f1:.2f}%")

  torch.save(model.state_dict(), 'encoder.pth')
  print(f'Model saved \n')


Entering the training loop 

Epoch 1, Batch 10, Loss: 0.2859
Epoch 1, Batch 20, Loss: 0.2405
Epoch 1, Batch 30, Loss: 0.1542
Epoch 1, Batch 40, Loss: 0.1449
Epoch 1, Batch 50, Loss: 0.0543
Epoch 1, Batch 60, Loss: 0.0839
Epoch 1, Batch 70, Loss: 0.1048
Epoch 1, Batch 80, Loss: 0.1339
Epoch 1, Batch 90, Loss: 0.1495
Epoch 1, Batch 100, Loss: 0.0782
Epoch 1, Batch 110, Loss: 0.1252
Epoch 1, Batch 120, Loss: 0.0720
Epoch 1, Batch 130, Loss: 0.2261
Epoch 1, Batch 140, Loss: 0.1323
Epoch 1, Batch 150, Loss: 0.1375
Epoch 1, Batch 160, Loss: 0.0874
Epoch 1, Batch 170, Loss: 0.1541
Epoch 1, Batch 180, Loss: 0.0707
Epoch 1, Batch 190, Loss: 0.1232
Epoch 1, Batch 200, Loss: 0.1244
Epoch 1, Batch 210, Loss: 0.1670
Epoch 1, Batch 220, Loss: 0.1014
Epoch 1, Batch 230, Loss: 0.1410
Epoch 1, Batch 240, Loss: 0.0380
Epoch 1, Batch 250, Loss: 0.0770
Epoch 1, Batch 260, Loss: 0.1096
Epoch 1, Batch 270, Loss: 0.1143
Epoch 1, Batch 280, Loss: 0.1283
Epoch 1, Batch 290, Loss: 0.1330
Epoch 1, Batch 300, Los

In [11]:
test_encodings = tokenizer(test_texts, max_length=MAX_LENGTH, padding=True, truncation=True, return_tensors='pt')
test_dataset = TextDataset(test_texts, [0]*len(test_texts), tokenizer, MAX_LENGTH)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate)

import csv

model.eval()
predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, _ = batch

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.round(torch.sigmoid(outputs))
        predictions.extend(preds.cpu().numpy())


predictions = [int(pred[0]) for pred in predictions]
output_file = 'predictions.csv'

with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'prediction'])
    for idx, prediction in enumerate(predictions):
        writer.writerow([idx, prediction])

print(f'Predictions saved to {output_file}')


Predictions saved to predictions.csv
